In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json

In [2]:
app = Flask(__name__)
received_data = []

In [3]:
import re, uuid, json, os
from datetime import datetime, timedelta
from typing import List, Dict, Any
from dateutil import tz
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build


def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/" + user.split("@")[0] + ".token"
    if not os.path.exists(token_path):
        return []

    try:
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
    except Exception:
        return []

    for event in events:
        attendee_list = []
        for attendee in event.get("attendees", []):
            attendee_list.append(attendee.get('email', "UNKNOWN"))

        start_time = event["start"].get("dateTime")
        end_time = event["end"].get("dateTime")
        events_list.append({
            "StartTime": start_time,
            "EndTime": end_time,
            "NumAttendees": len(set(attendee_list)),
            "Attendees": list(set(attendee_list)),
            "Summary": event.get("summary", "No Summary")
        })
    return events_list


class MeetingSchedulerAgent:
    def __init__(self, model_client=None, model_path=None):
        self.model_client = model_client
        self.model_path = model_path
        self.timezone = tz.gettz("Asia/Kolkata")

    def extract_meeting_details(self, email_content: str, reference_datetime: str, parsed: Dict[str, Any]):
        weekday_map = {
            "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
            "friday": 4, "saturday": 5, "sunday": 6
        }

        next_weekday_match = re.search(r"\bnext\s+(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b", email_content.lower())
        weekday_match = re.search(r"\b(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b", email_content.lower())
        next_week_match = re.search(r"\bnext week\b", email_content.lower())
        time_match = re.search(r"(\d{1,2}):(\d{2})", email_content)

        try:
            raw_duration = parsed.get("meeting_duration")
            duration_int = int(raw_duration)
            if duration_int <= 0 or duration_int > 1440:
                raise ValueError("Invalid duration from LLM")
            duration_mins = str(duration_int)
        except Exception:
            duration_mins = "30"

        try:
            ref_dt = datetime.fromisoformat(reference_datetime)
        except ValueError:
            ref_dt = datetime.strptime(reference_datetime, "%d-%m-%YT%H:%M:%S")
        ref_dt = ref_dt.replace(tzinfo=self.timezone)

        if next_week_match:
            target_weekday = 0
            if weekday_match:
                target_weekday = weekday_map[weekday_match.group(1)]
            days_ahead = (target_weekday - ref_dt.weekday() + 7) % 7 + 7
            start_dt = ref_dt + timedelta(days=days_ahead)
        elif next_weekday_match or weekday_match:
            target_weekday = weekday_map[(next_weekday_match or weekday_match).group(1)]
            days_ahead = (target_weekday - ref_dt.weekday() + 7) % 7
            if next_weekday_match:
                days_ahead += 7
            start_dt = ref_dt + timedelta(days=days_ahead)
        else:
            start_dt = ref_dt

        if time_match:
            hour = int(time_match.group(1))
            minute = int(time_match.group(2))
            start_dt = start_dt.replace(hour=hour, minute=minute, second=0)
            end_dt = start_dt.replace(hour=23, minute=59, second=59)
        else:
            start_dt = start_dt.replace(hour=0, minute=0, second=0)
            end_dt = start_dt.replace(hour=23, minute=59, second=59)

        return start_dt.isoformat(), end_dt.isoformat(), duration_mins

    def parse_email(self, email_text: str) -> Dict[str, Any]:
        if not self.model_client:
            raise ValueError("No model client provided for LLM inference.")

        prompt = f"""
You are an intelligent meeting assistant.

Extract the following details from the meeting email:
1. List of participant emails (append `@amd.com` if only names).
2. Meeting duration in minutes.
3. Subject of the meeting.
4. Any time constraint like 'next week', 'Monday at 10am', etc.

If email mentions only the day of the meeting, assume start time is 00:00:00 and end time is 23:59:59.
If email mentions the time of the meeting, take that as start and end of the day as end time.

If duration is not explicitly mentioned, infer a suitable meeting duration.

Respond ONLY in JSON with this format:
{{
  "participants": [...],
  "time_constraints": "...",
  "meeting_duration": ...,
  "subject": "..."
}}

Email: {email_text}
"""
        response = self.model_client.chat.completions.create(
            model=self.model_path,
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return json.loads(response.choices[0].message.content)

    def is_conflict(self, events: List[Dict], start: datetime, end: datetime):
        for event in events:
            ev_start = datetime.fromisoformat(event["StartTime"])
            ev_end = datetime.fromisoformat(event["EndTime"])
            if not (end <= ev_start or start >= ev_end):
                return True
        return False

    def process_meeting_input(self, meeting_dict: Dict[str, Any]):
        parsed = self.parse_email(meeting_dict["EmailContent"])

        start, end, duration = self.extract_meeting_details(
            meeting_dict["EmailContent"],
            meeting_dict["Datetime"],
            parsed
        )

        subject = meeting_dict.get("Subject")
        if not subject:
            subject = parsed.get("subject", "Meeting")

        meeting_dict.update({
            "Start": start,
            "End": end,
            "Duration_mins": duration,
            "Subject": subject
        })
        return meeting_dict

    def append_event_to_output(self, email: str, events_map: Dict[str, List[Dict]], event: Dict[str, Any]):
        if email not in events_map:
            events_map[email] = []
        events_map[email].append(event)

    def schedule(self, meeting_payload: Dict[str, Any]):
        dt_str = meeting_payload.get("Datetime") or datetime.now().isoformat()

        meeting_dict = {
            "From": meeting_payload["From"],
            "Attendees": meeting_payload["Attendees"],
            "EmailContent": meeting_payload["EmailContent"],
            "Datetime": dt_str,
            "Location": meeting_payload.get("Location", ""),
            "Subject": meeting_payload.get("Subject")  # May be None
        }

        processed = self.process_meeting_input(meeting_dict.copy())
        display_processed = processed.copy()
        display_processed["Attendees"] = [a["email"] if isinstance(a, dict) else a for a in display_processed["Attendees"]]

        return {
            "processed_input": display_processed,
            "scheduled_output": self._schedule_internal(processed)
        }

    def _schedule_internal(self, meeting_dict: Dict[str, Any]):
        from_email = meeting_dict["From"]
        attendees = meeting_dict["Attendees"]
        start, end = meeting_dict["Start"], meeting_dict["End"]
        emails = [from_email] + [a["email"] if isinstance(a, dict) else a for a in attendees]
        events_map = {}

        for email in emails:
            events_map[email] = retrive_calendar_events(email, start, end)

        try:
            duration = timedelta(minutes=int(meeting_dict["Duration_mins"]))
            if duration.total_seconds() > 12 * 60 * 60:
                raise ValueError("Unreasonable duration")
        except Exception:
            duration = timedelta(minutes=30)

        ref_date = datetime.fromisoformat(start)
        if ref_date.hour == 0 and ref_date.minute == 0:
            ref_date = ref_date.replace(hour=9, minute=0, second=0)

        while True:
            for hour in range(9, 18):
                slot_start = ref_date.replace(hour=hour, minute=0, second=0)
                slot_end = slot_start + duration
                if slot_end.hour > 18 or slot_end.day != slot_start.day:
                    continue
                if not any(self.is_conflict(events, slot_start, slot_end) for events in events_map.values()):
                    event = {
                        "StartTime": slot_start.isoformat(),
                        "EndTime": slot_end.isoformat(),
                        "NumAttendees": len(attendees) + 1,
                        "Attendees": emails,
                        "Summary": meeting_dict["Subject"]  # Same as subject
                    }
                    for email in emails:
                        self.append_event_to_output(email, events_map, event)

                    return {
                        "Request_id": str(uuid.uuid4()),
                        "Datetime": meeting_dict["Datetime"],
                        "Location": meeting_dict.get("Location", ""),
                        "From": from_email,
                        "Attendees": [{"email": email, "events": events} for email, events in events_map.items()],
                        "Subject": meeting_dict["Subject"],
                        "EmailContent": meeting_dict["EmailContent"],
                        "EventStart": event["StartTime"],
                        "EventEnd": event["EndTime"],
                        "Duration_mins": meeting_dict["Duration_mins"]
                    }

            ref_date += timedelta(days=1)
            ref_date = ref_date.replace(hour=9, minute=0, second=0)

In [4]:
from openai import OpenAI
MODEL_PATH = "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct"
client = OpenAI(api_key="NULL", base_url="http://localhost:4000/v1", timeout=None, max_retries=0)
agent = MeetingSchedulerAgent(client, MODEL_PATH)

In [5]:
def your_meeting_assistant(data): 
    meeting_output = agent.schedule(data)
    data["processed"] = meeting_output.get('processed_input')
    data["output"] = meeting_output.get('scheduled_output')
    return data

In [6]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=6000)

In [7]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6000
 * Running on http://129.212.176.234:6000
Press CTRL+C to quit



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "09-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}


127.0.0.1 - - [13/Jul/2025 05:38:18] "POST /receive HTTP/1.1" 200 -





 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "09-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",
  "processed": {
    "From": "userone.amd@gmail.com",
    "Attendees": [
      "usertwo.amd@gmail.com",
      "userthree.amd@gmail.com"
    ],
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "Subject": "Agentic AI Project Status Update",
    "Start": "2025-07-10T00:00:00+05:30",
    "End": "2025-07-10T23:59:59+05:30",
    "Duration_mins": "30"
  },
  "output": {
    "Request_id": "094e40f7-019c-4

In [9]:
import requests
import json

url = "http://localhost:6000/receive"
payload = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {"email": "usertwo.amd@gmail.com"},
        {"email": "userthree.amd@gmail.com"}
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

response = requests.post(url, json=payload)

print("Response:", json.dumps(response.json(), indent=2))

Response: {
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Datetime": "09-07-2025T12:34:55",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",
  "From": "userone.amd@gmail.com",
  "Location": "IIT Mumbai",
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Subject": "Agentic AI Project Status Update",
  "output": {
    "Attendees": [
      {
        "email": "userone.amd@gmail.com",
        "events": [
          {
            "Attendees": [],
            "EndTime": "2025-07-10T09:00:00+05:30",
            "NumAttendees": 0,
            "StartTime": "2025-07-09T18:00:00+05:30",
            "Summary": "Off Hours"
          },
          {
            "Attendees": [],
            "EndTime": "2025-07-11T09:00:00+05:30",
            "NumAttendees": 0,
            "StartTime": "2025-07-10T18:00:00+05:30",
            "Summary": "Off Hours"
 